In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.io import decode_jpeg,read_file
from tensorflow.data import Dataset
from tensorflow.keras.layers import Conv2D,Dropout,MaxPool2D,Dense,Flatten,Rescaling,RandomFlip,RandomRotation,Resizing, BatchNormalization, AveragePooling1D
from keras import metrics
from tensorflow.keras.metrics import Metric
from tensorflow.keras.optimizers import Nadam
import glob
import os
import datetime
import matplotlib.pyplot as plt

2023-09-07 13:45:54.253535: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 13:45:56.630020: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-09-07 13:45:56.630139: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-09-07 13:45:56.630151: W tensorfl

In [6]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Physical devices cannot be modified after being initialized


In [4]:
# path_data = "./ambrosia+/train/1800006/*.jpg"
# files = glob.glob(path_data,recursive=True)
# files = files[230:]
# print(len(files))
# for i in files:
#     os.remove(i)

In [40]:
path_data = "./ambrosia+/train/"
BATCH_SIZE = 72
dataset = tf.keras.utils.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(225,225)
)
IRL_path = "./ambrosia+/testIRL/"
dataset_IRL = tf.keras.utils.image_dataset_from_directory(
    IRL_path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(225,225)
)
test_path = "./ambrosia+/test/"
dataset_test = tf.keras.utils.image_dataset_from_directory(
    test_path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(225,225))

model = keras.Sequential()
model.add(Conv2D(16,3,input_shape=(225,225,3),padding="same",activation="relu"))
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(16,3,activation="relu",padding="same"))
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(16,3,activation="relu",padding="same"))
model.add(MaxPool2D(2,padding="same"))
model.add(Flatten())
model.add(Dense(16,activation="relu",kernel_regularizer=tf.keras.regularizers.L1L2(l1=0.07, l2=0.05)))
model.add(Dropout(0.45))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer=tf.keras.optimizers.experimental.SGD(learning_rate=0.0017,momentum=0.004),loss=keras.losses.BinaryCrossentropy(),metrics=["accuracy"])
model.fit(dataset,epochs=10)
print("")
model.evaluate(dataset_test)
print("")
model.evaluate(dataset_IRL)
model.save_weights("ambrosia_next_sgd_nesterov_drop40")

Found 8047 files belonging to 2 classes.
Found 42 files belonging to 2 classes.
Found 935 files belonging to 2 classes.
Epoch 1/10


2023-07-04 14:10:21.090078: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [8047]
	 [[{{node Placeholder/_4}}]]
2023-07-04 14:10:21.090444: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [8047]
	 [[{{node Placeholder/_4}}]]


112/112 [==============================] - 16s 126ms/step - loss: 81.6781 - accuracy: 0.5576
Epoch 2/10
112/112 [==============================] - 15s 130ms/step - loss: 5.7804 - accuracy: 0.5652
Epoch 3/10
112/112 [==============================] - 15s 129ms/step - loss: 1.7175 - accuracy: 0.6286
Epoch 4/10
112/112 [==============================] - 15s 129ms/step - loss: 1.5856 - accuracy: 0.7603
Epoch 5/10
112/112 [==============================] - 15s 129ms/step - loss: 1.5197 - accuracy: 0.7993
Epoch 6/10
112/112 [==============================] - 15s 130ms/step - loss: 1.5040 - accuracy: 0.8048
Epoch 7/10
112/112 [==============================] - 15s 129ms/step - loss: 1.4930 - accuracy: 0.8173
Epoch 8/10
112/112 [==============================] - 15s 130ms/step - loss: 1.4772 - accuracy: 0.8184
Epoch 9/10
112/112 [==============================] - 15s 129ms/step - loss: 1.5556 - accuracy: 0.8232
Epoch 10/10
112/112 [==============================] - 15s 130ms/step - loss: 1.522

2023-07-04 14:12:56.287073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [935]
	 [[{{node Placeholder/_4}}]]
2023-07-04 14:12:56.287609: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [935]
	 [[{{node Placeholder/_4}}]]


13/13 [==============================] - 2s 44ms/step - loss: 1.4262 - accuracy: 0.8738



2023-07-04 14:12:57.846684: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [42]
	 [[{{node Placeholder/_4}}]]
2023-07-04 14:12:57.847117: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [42]
	 [[{{node Placeholder/_4}}]]


1/1 [==============================] - 1s 552ms/step - loss: 1.8536 - accuracy: 0.6905


In [ ]:
BATCH_SIZE = 100

dataset_train_path = "./dataset_ambrosia/train"
dataset_train = tf.keras.utils.image_dataset_from_directory(
    dataset_train_path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    seed=42,
    validation_split=0.3,
    subset='validation',
    image_size=(224,224))

dataset_val_path = "./dataset_ambrosia/val"
dataset_val = tf.keras.utils.image_dataset_from_directory(
    dataset_val_path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(224,224))


test1_path = "./dataset_ambrosia/test"
dataset_test = tf.keras.utils.image_dataset_from_directory(
    test1_path,
    labels="inferred",
    label_mode="int",
    color_mode="rgb",
    batch_size=BATCH_SIZE,
    image_size=(224,224))

# test2_path = "./dataset_ambrosia/test_2"
# dataset_test2 = tf.keras.utils.image_dataset_from_directory(
#     test2_path,
#     labels="inferred",
#     label_mode="int",
#     color_mode="rgb",
#     batch_size=BATCH_SIZE,
#     image_size=(350,350))

# test2_crop_path = "./dataset_ambrosia/test_2_crop"
# dataset_test2_crop = tf.keras.utils.image_dataset_from_directory(
#     test2_crop_path,
#     labels="inferred",
#     label_mode="int",
#     color_mode="rgb",
#     batch_size=BATCH_SIZE,
#     image_size=(350,350))

model = keras.Sequential()
model.add(Conv2D(64,2,input_shape=(224,224,3),padding="same",activation="relu"))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(64,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(128,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(128,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(256,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(256,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(512,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(512,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(1024,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))# test
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same"))
model.add(Conv2D(1024,3,activation="relu",padding="same", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(tf.keras.layers.BatchNormalization())
model.add(MaxPool2D(2,padding="same")) # end test
model.add(Flatten())
model.add(Dense(1024,activation="relu", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
# model.add(Dropout(0.45))
model.add(Dense(2048,activation="relu", kernel_regularizer=tf.keras.regularizers.L2(l2=0.05)))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer=keras.optimizers.experimental.RMSprop(learning_rate=0.000001),loss=keras.losses.BinaryCrossentropy(),metrics=["accuracy"])

epochs = 40
history = model.fit(dataset_train, epochs=epochs)

acc = history.history['accuracy']
loss = history.history['loss']
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.scatter(epochs_range, acc, label='Training Accuracy')

plt.plot(epochs_range, loss, label='Training Loss')
plt.scatter(epochs_range, loss, label='Training Loss')
plt.title('Accuracy')
plt.show()

print("val:")
model.evaluate(dataset_val)
print("test1:")
model.evaluate(dataset_test)
# print("test2:")
# model.evaluate(dataset_test2)
# print("test2_crop:")
# model.evaluate(dataset_test2_crop)
print("")
model.save('ambrosiaTEST.h5')

Found 33984 files belonging to 2 classes.
Using 10195 files for validation.
Found 1274 files belonging to 2 classes.
Found 104 files belonging to 2 classes.
Epoch 1/40
102/102 [==============================] - 168s 1s/step - loss: 282.0865 - accuracy: 0.5923
Epoch 2/40
102/102 [==============================] - 213s 2s/step - loss: 279.7760 - accuracy: 0.7183
Epoch 3/40
102/102 [==============================] - 171s 2s/step - loss: 277.4941 - accuracy: 0.7653
Epoch 4/40
  6/102 [>.............................] - ETA: 2:28 - loss: 276.2926 - accuracy: 0.7850

In [2]:
import torch 
import torchvision
import torch.nn as nn

In [3]:
def get_optim(parameters, optim_name: str, lr):
    optim = {
        'SGD': torch.optim.SGD, 
        'AdamW': torch.optim.AdamW, 
        'NAdam': torch.optim.NAdam, 
        'RMSprop': torch.optim.RMSprop, 
        'Adam': torch.optim.Adam,
    }
    optimizer = optim(parameters(), lr)
    return optimizer

In [4]:
model = nn.Linear(10,1)

# params = model.parameters()

lr = 0.0001
optimizers = 'Adam'

# model = torchvision.models.resnet18

optimizer = get_optim(model.parameters(), optimizers, lr)
print(optimizer)

TypeError: 'generator' object is not callable

In [8]:
model = torchvision.models.resnet18
optimizer = torch.optim.RMSprop(lr=0.01, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False, foreach=None, maximize=False, differentiable=False)
print(optimizer)

TypeError: __init__() missing 1 required positional argument: 'params'